In [1]:
import sys
import os
import warnings

sys.path.append(
    os.path.abspath(os.path.join(os.path.dirname(os.path.realpath("__file__")), ".."))
)

warnings.simplefilter(action="ignore", category=FutureWarning)

In [2]:
import pandas as pd
from datetime import date

import data_loader.yahoo_finance as yahoo_finance
import utils.downloading_utils as downloading_utils
import utils.sql_utils as sql_utils
import utils.mapping as mapping

import handyman.holdings as holdings
import handyman.prices as prices
import handyman.company_info as company_info
from utils.yfinance_utils import (
    pull_prices,
    pull_financials,
    create_client,
    pull_officers,
    pull_info,
)
from utils.database_utils import find_missing_tickers

# Download Holdings Files

In [19]:
etf_urls = mapping.etf_urls
data_path = os.path.join(
    os.path.abspath(os.path.join(os.path.dirname(os.path.realpath("__file__")), "..")),
    "Data",
)

all_indices = pd.DataFrame()
for fund_name, url in etf_urls.items():
    print(fund_name)
    df_temp = downloading_utils.download_holdings(
        fund_name, url, download_folder=data_path
    )
    all_indices = pd.concat([all_indices, df_temp])

S&P 500
Russell 1000


In [22]:
sql_utils.write_sql_table(
    database_name="CODE_CAPITAL", table_name="holdings", df=all_indices, overwrite=False
)

In [ ]:
# all_indices = holdings.get_index_holdings(
#     # indices=['S&P 500'],
#     tickers=None,
#     start_date="2025-12-26",
# )

# all_indices

,DATE,INDEX,TICKER,NAME,MARKET_VALUE,WEIGHT,QUANTITY,PRICE,LOCATION,EXCHANGE,CURRENCY,FX_RATE
0,2025-12-26,S&P 500,A,AGILENT TECHNOLOGIES INC,5.085527e+08,0.000663,3676639.0,138.32,United States,New York Stock Exchange Inc.,USD,1.0
1,2025-12-26,S&P 500,AAPL,APPLE INC,5.247041e+10,0.068401,191630732.0,273.81,United States,NASDAQ,USD,1.0
2,2025-12-26,S&P 500,ABBV,ABBVIE INC,5.269254e+09,0.006869,22920763.0,229.89,United States,New York Stock Exchange Inc.,USD,1.0
3,2025-12-26,S&P 500,ABNB,AIRBNB INC CLASS A,7.544141e+08,0.000983,5515529.0,136.78,United States,NASDAQ,USD,1.0
4,2025-12-26,S&P 500,ABT,ABBOTT LABORATORIES,2.814588e+09,0.003669,22550985.0,124.81,United States,New York Stock Exchange Inc.,USD,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1506,2025-12-26,Russell 1000,ZG,ZILLOW GROUP INC CLASS A,2.376151e+06,0.000051,34512.0,68.85,United States,NASDAQ,USD,1.0
1507,2025-12-26,Russell 1000,ZION,ZIONS BANCORPORATION,6.441643e+06,0.000138,108172.0,59.55,United States,NASDAQ,USD,1.0
1508,2025-12-26,Russell 1000,ZM,ZOOM COMMUNICATIONS INC CLASS A,1.682329e+07,0.000360,191631.0,87.79,United States,NASDAQ,USD,1.0
1509,2025-12-26,Russell 1000,ZS,ZSCALER INC,1.641732e+07,0.000352,71504.0,229.60,United States,NASDAQ,USD,1.0


In [ ]:
# sql_utils.delete_sql_rows(
#     database_name="CODE_CAPITAL",
#     table_name="holdings",
#     where_clause="date is '2025-12-26 00:00:00'"
# )

# Store Company Data

## Company Prices

In [9]:
all_prices = pull_prices(tickers=all_indices["TICKER"].unique())

sql_utils.write_sql_table(
    table_name="prices", database_name="CODE_CAPITAL", df=all_prices, overwrite=False
)

2025-12-27 11:52:21 - INFO - Loading Prices


In [ ]:
adjusted = list(
    all_prices[(all_prices["DIVIDENDS"] > 0) | (all_prices["STOCK_SPLITS"] > 0)][
        "TICKER"
    ].unique()
)
print(adjusted)

if adjusted:
    adj_secs = "' ,'".join(adjusted)
    sql_utils.delete_sql_rows(
        database_name="CODE_CAPITAL",
        table_name="prices",
        where_clause=f"ticker in ('{adj_secs}')",
    )
    adjusted_prices = pull_prices(tickers=adjusted)

    sql_utils.write_sql_table(
        table_name="prices",
        database_name="CODE_CAPITAL",
        df=adjusted_prices,
        overwrite=False,
    )

2025-12-27 11:52:59 - INFO - Loading Prices


In [14]:
prices.get_prices(tickers=["AAPL", "MSFT"]).sort_index(ascending=False)

TICKER,AAPL,MSFT
DATE,,
2025-12-26,273.399994,487.709991
2025-12-24,273.809998,488.019989
2025-12-23,272.359985,486.850006
2025-12-22,270.970001,484.920013
2025-12-19,273.670013,485.920013
...,...,...
2000-01-07,0.746027,34.036129
2000-01-06,0.712287,33.597080
2000-01-05,0.779767,34.761539


## Financials

In [15]:
run = False

today = date.today()
if today.weekday() == 0:
    all_financial_annual = pull_financials(
        tickers=all_indices["TICKER"].unique(), annual=True, statement_type="financial"
    )
    all_financial_quarterly = pull_financials(
        tickers=all_indices["TICKER"].unique(), annual=False, statement_type="financial"
    )
    all_balancesheet_annual = pull_financials(
        tickers=all_indices["TICKER"].unique(),
        annual=True,
        statement_type="balance_sheet",
    )
    all_balancesheet_quarterly = pull_financials(
        tickers=all_indices["TICKER"].unique(),
        annual=False,
        statement_type="balance_sheet",
    )
    all_income_annual = pull_financials(
        tickers=all_indices["TICKER"].unique(),
        annual=True,
        statement_type="income_statement",
    )
    all_income_quarterly = pull_financials(
        tickers=all_indices["TICKER"].unique(),
        annual=False,
        statement_type="income_statement",
    )
    all_cashflow_annual = pull_financials(
        tickers=all_indices["TICKER"].unique(), annual=True, statement_type="cashflow"
    )
    all_cashflow_quarterly = pull_financials(
        tickers=all_indices["TICKER"].unique(), annual=False, statement_type="cashflow"
    )
    run = True

else:
    missing = find_missing_tickers("financial_annual", all_indices["TICKER"].unique())
    print(missing)
    if missing:
        all_financial_annual = pull_financials(
            tickers=missing, annual=True, statement_type="financial"
        )
        all_financial_quarterly = pull_financials(
            tickers=missing, annual=False, statement_type="financial"
        )
        all_balancesheet_annual = pull_financials(
            tickers=missing, annual=True, statement_type="balance_sheet"
        )
        all_balancesheet_quarterly = pull_financials(
            tickers=missing, annual=False, statement_type="balance_sheet"
        )
        all_income_annual = pull_financials(
            tickers=missing, annual=True, statement_type="income_statement"
        )
        all_income_quarterly = pull_financials(
            tickers=missing, annual=False, statement_type="income_statement"
        )
        all_cashflow_annual = pull_financials(
            tickers=missing, annual=True, statement_type="cashflow"
        )
        all_cashflow_quarterly = pull_financials(
            tickers=missing, annual=False, statement_type="cashflow"
        )
        run = True

2025-12-27 11:53:50 - INFO - Loading Company Financial


['FRMI']


ValueError: No objects to concatenate

In [ ]:
if run:
    sql_utils.write_sql_table(
        table_name="financial_annual",
        database_name="CODE_CAPITAL",
        df=all_financial_annual,
        overwrite=False,
    )
    sql_utils.write_sql_table(
        table_name="financial_quarterly",
        database_name="CODE_CAPITAL",
        df=all_financial_quarterly,
        overwrite=False,
    )
    sql_utils.write_sql_table(
        table_name="balancesheet_annual",
        database_name="CODE_CAPITAL",
        df=all_balancesheet_annual,
        overwrite=False,
    )
    sql_utils.write_sql_table(
        table_name="balancesheet_quarterly",
        database_name="CODE_CAPITAL",
        df=all_balancesheet_quarterly,
        overwrite=False,
    )
    sql_utils.write_sql_table(
        table_name="incomestatement_annual",
        database_name="CODE_CAPITAL",
        df=all_income_annual,
        overwrite=False,
    )
    sql_utils.write_sql_table(
        table_name="incomestatement_quarterly",
        database_name="CODE_CAPITAL",
        df=all_income_quarterly,
        overwrite=False,
    )
    sql_utils.write_sql_table(
        table_name="cashflow_annual",
        database_name="CODE_CAPITAL",
        df=all_cashflow_annual,
        overwrite=False,
    )
    sql_utils.write_sql_table(
        table_name="cashflow_quarterly",
        database_name="CODE_CAPITAL",
        df=all_cashflow_quarterly,
        overwrite=False,
    )

In [10]:
query = """
SELECT *
FROM cashflow_annual
WHERE "TICKER" IN ('AAPL')
ORDER BY DATE DESC, TICKER
"""

sql_utils.read_sql_table("CODE_CAPITAL", query=query)

,DATE,TICKER,REPORT_DATE,FREE_CASH_FLOW,REPURCHASE_OF_CAPITAL_STOCK,REPAYMENT_OF_DEBT,ISSUANCE_OF_DEBT,CAPITAL_EXPENDITURE,INTEREST_PAID_SUPPLEMENTAL_DATA,INCOME_TAX_PAID_SUPPLEMENTAL_DATA,...,EXCESS_TAX_BENEFIT_FROM_STOCK_BASED_COMPENSATION,INTEREST_PAID_CFO,CASH_FLOWSFROMUSEDIN_OPERATING_ACTIVITIES_DIRECT,CLASSESOF_CASH_PAYMENTS,OTHER_CASH_PAYMENTSFROM_OPERATING_ACTIVITIES,PAYMENTSON_BEHALFOF_EMPLOYEES,CLASSESOF_CASH_RECEIPTSFROM_OPERATING_ACTIVITIES,OTHER_CASH_RECEIPTSFROM_OPERATING_ACTIVITIES,INTEREST_RECEIVED_CFO,CHANGE_IN_DIVIDEND_PAYABLE
0,2025-12-22,AAPL,2025-09-30 00:00:00,9.876700e+10,-9.071100e+10,-1.093200e+10,4.481000e+09,-1.271500e+10,NaN,4.336900e+10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2025-12-22,AAPL,2024-09-30 00:00:00,1.088070e+11,-9.494900e+10,-9.958000e+09,0.000000e+00,-9.447000e+09,NaN,2.610200e+10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2025-12-22,AAPL,2023-09-30 00:00:00,9.958400e+10,-7.755000e+10,-1.115100e+10,5.228000e+09,-1.095900e+10,3.803000e+09,1.867900e+10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2025-12-22,AAPL,2022-09-30 00:00:00,1.114430e+11,-8.940200e+10,-9.543000e+09,5.465000e+09,-1.070800e+10,2.865000e+09,1.957300e+10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2025-12-22,AAPL,2021-09-30 00:00:00,NaN,NaN,NaN,NaN,NaN,2.687000e+09,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2025-12-08,AAPL,2025-09-30 00:00:00,9.876700e+10,-9.071100e+10,-1.093200e+10,4.481000e+09,-1.271500e+10,NaN,4.336900e+10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2025-12-08,AAPL,2024-09-30 00:00:00,1.088070e+11,-9.494900e+10,-9.958000e+09,0.000000e+00,-9.447000e+09,NaN,2.610200e+10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2025-12-08,AAPL,2023-09-30 00:00:00,9.958400e+10,-7.755000e+10,-1.115100e+10,5.228000e+09,-1.095900e+10,3.803000e+09,1.867900e+10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2025-12-08,AAPL,2022-09-30 00:00:00,1.114430e+11,-8.940200e+10,-9.543000e+09,5.465000e+09,-1.070800e+10,2.865000e+09,1.957300e+10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2025-12-08,AAPL,2021-09-30 00:00:00,NaN,NaN,NaN,NaN,NaN,2.687000e+09,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Company Info

In [16]:
run = False

today = date.today()
if today.weekday() == 0:
    client = create_client(tickers=all_indices["TICKER"].unique())
    all_info = pull_info(tickers=all_indices["TICKER"].unique(), client=client)
    all_officers = pull_officers(tickers=all_indices["TICKER"].unique(), client=client)
    run = True
else:
    missing = find_missing_tickers("company_info", all_indices["TICKER"].unique())
    print(missing)
    if missing:
        client = create_client(tickers=missing)
        all_info = pull_info(tickers=missing, client=client)
        all_officers = pull_officers(tickers=missing, client=client)
        run = True

[]


In [ ]:
if run:
    sql_utils.write_sql_table(
        table_name="company_info",
        database_name="CODE_CAPITAL",
        df=all_info,
        overwrite=False,
    )
    sql_utils.write_sql_table(
        table_name="officers",
        database_name="CODE_CAPITAL",
        df=all_officers,
        overwrite=False,
    )

In [2]:
company_info.get_company_info(tickers=None)

NameError: name 'company_info' is not defined

In [20]:
info_df = company_info.get_company_info(tickers=None)

# Read the data from the database into a pandas DataFrame
cachedf = sql_utils.read_sql_table("CODE_CAPITAL", table_name="address")

missing_address = info_df[~(info_df["ADDRESS1"].isin(cachedf["ADDRESS1"].unique()))]

missing_address

,DATE,TICKER,ADDRESS1,CITY,STATE,ZIP,COUNTRY,PHONE,WEBSITE,INDUSTRY,INDUSTRYKEY,INDUSTRYDISP,SECTOR,SECTORKEY,SECTORDISP,LONGBUSINESSSUMMARY,FULLTIMEEMPLOYEES,AUDITRISK,BOARDRISK,COMPENSATIONRISK,SHAREHOLDERRIGHTSRISK,OVERALLRISK,GOVERNANCEEPOCHDATE,COMPENSATIONASOFEPOCHDATE,IRWEBSITE,EXECUTIVETEAM,MAXAGE,PRICEHINT,PREVIOUSCLOSE,OPEN,DAYLOW,DAYHIGH,REGULARMARKETPREVIOUSCLOSE,REGULARMARKETOPEN,REGULARMARKETDAYLOW,REGULARMARKETDAYHIGH,DIVIDENDRATE,DIVIDENDYIELD,EXDIVIDENDDATE,PAYOUTRATIO,FIVEYEARAVGDIVIDENDYIELD,BETA,TRAILINGPE,FORWARDPE,VOLUME,REGULARMARKETVOLUME,AVERAGEVOLUME,AVERAGEVOLUME10DAYS,AVERAGEDAILYVOLUME10DAY,BID,ASK,BIDSIZE,ASKSIZE,MARKETCAP,FIFTYTWOWEEKLOW,FIFTYTWOWEEKHIGH,ALLTIMEHIGH,ALLTIMELOW,PRICETOSALESTRAILING12MONTHS,FIFTYDAYAVERAGE,TWOHUNDREDDAYAVERAGE,TRAILINGANNUALDIVIDENDRATE,TRAILINGANNUALDIVIDENDYIELD,CURRENCY,TRADEABLE,ENTERPRISEVALUE,PROFITMARGINS,FLOATSHARES,SHARESOUTSTANDING,SHARESSHORT,SHARESSHORTPRIORMONTH,SHARESSHORTPREVIOUSMONTHDATE,DATESHORTINTEREST,SHARESPERCENTSHARESOUT,HELDPERCENTINSIDERS,HELDPERCENTINSTITUTIONS,SHORTRATIO,SHORTPERCENTOFFLOAT,IMPLIEDSHARESOUTSTANDING,BOOKVALUE,PRICETOBOOK,LASTFISCALYEAREND,NEXTFISCALYEAREND,MOSTRECENTQUARTER,EARNINGSQUARTERLYGROWTH,NETINCOMETOCOMMON,TRAILINGEPS,FORWARDEPS,LASTSPLITFACTOR,LASTSPLITDATE,ENTERPRISETOREVENUE,ENTERPRISETOEBITDA,52WEEKCHANGE,SANDP52WEEKCHANGE,LASTDIVIDENDVALUE,LASTDIVIDENDDATE,QUOTETYPE,CURRENTPRICE,TARGETHIGHPRICE,TARGETLOWPRICE,TARGETMEANPRICE,TARGETMEDIANPRICE,RECOMMENDATIONMEAN,RECOMMENDATIONKEY,NUMBEROFANALYSTOPINIONS,TOTALCASH,TOTALCASHPERSHARE,EBITDA,TOTALDEBT,QUICKRATIO,CURRENTRATIO,TOTALREVENUE,DEBTTOEQUITY,REVENUEPERSHARE,RETURNONASSETS,RETURNONEQUITY,GROSSPROFITS,FREECASHFLOW,OPERATINGCASHFLOW,EARNINGSGROWTH,REVENUEGROWTH,GROSSMARGINS,EBITDAMARGINS,OPERATINGMARGINS,FINANCIALCURRENCY,SYMBOL,LANGUAGE,REGION,TYPEDISP,QUOTESOURCENAME,TRIGGERABLE,CUSTOMPRICEALERTCONFIDENCE,SHORTNAME,LONGNAME,MARKETSTATE,ESGPOPULATED,CORPORATEACTIONS,POSTMARKETTIME,REGULARMARKETTIME,EXCHANGE,MESSAGEBOARDID,EXCHANGETIMEZONENAME,EXCHANGETIMEZONESHORTNAME,GMTOFFSETMILLISECONDS,MARKET,FULLEXCHANGENAME,AVERAGEDAILYVOLUME3MONTH,FIFTYTWOWEEKLOWCHANGE,FIFTYTWOWEEKLOWCHANGEPERCENT,FIFTYTWOWEEKRANGE,FIFTYTWOWEEKHIGHCHANGE,FIFTYTWOWEEKHIGHCHANGEPERCENT,FIFTYTWOWEEKCHANGEPERCENT,DIVIDENDDATE,EARNINGSTIMESTAMP,EARNINGSTIMESTAMPSTART,EARNINGSTIMESTAMPEND,EARNINGSCALLTIMESTAMPSTART,EARNINGSCALLTIMESTAMPEND,ISEARNINGSDATEESTIMATE,EPSTRAILINGTWELVEMONTHS,EPSFORWARD,EPSCURRENTYEAR,PRICEEPSCURRENTYEAR,FIFTYDAYAVERAGECHANGE,FIFTYDAYAVERAGECHANGEPERCENT,TWOHUNDREDDAYAVERAGECHANGE,TWOHUNDREDDAYAVERAGECHANGEPERCENT,SOURCEINTERVAL,EXCHANGEDATADELAYEDBY,PREVNAME,NAMECHANGEDATE,AVERAGEANALYSTRATING,CRYPTOTRADEABLE,HASPREPOSTMARKETDATA,FIRSTTRADEDATEMILLISECONDS,POSTMARKETCHANGEPERCENT,POSTMARKETPRICE,POSTMARKETCHANGE,REGULARMARKETCHANGE,REGULARMARKETDAYRANGE,REGULARMARKETCHANGEPERCENT,REGULARMARKETPRICE,DISPLAYNAME,TRAILINGPEGRATIO,IPOEXPECTEDDATE,ADDRESS2,FAX,INDUSTRYSYMBOL,PEGRATIO


In [18]:
from utils import geo

df = geo.geocode_dataframe(missing_address, cache_df=cachedf, delay=0.5)

df = df[["ADDRESS1", "CITY", "COUNTRY", "LAT", "LON"]].dropna(subset="LAT")

df

Geocoding: 100%|██████████| 6/6 [00:13<00:00,  2.29s/it]


,ADDRESS1,CITY,COUNTRY,LAT,LON
15,702 South West 8th Street,Bentonville,United States,36.372861,-94.208787
449,LyondellBasell Williams Tower,Houston,United States,29.758938,-95.367697
466,909 Third Avenue,New York,United States,40.758283,-73.967866
1028,702 South West 8th Street,Bentonville,United States,36.372861,-94.208787
1460,LyondellBasell Williams Tower,Houston,United States,29.758938,-95.367697
1482,909 Third Avenue,New York,United States,40.758283,-73.967866


In [19]:
sql_utils.write_sql_table(
    database_name="CODE_CAPITAL", table_name="address", df=df, overwrite=False
)